Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Adding L2 regularization to logistic model

In [7]:
batch_size = 128

strength_of_regularizer = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Adding Regularization
  L2_regularizer = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))

  # Add the regularization term to the loss.
  loss += strength_of_regularizer * L2_regularizer
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.962673
Minibatch accuracy: 10.2%
Validation accuracy: 10.2%
Minibatch loss at step 500: 2.895975
Minibatch accuracy: 75.0%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 1.702831
Minibatch accuracy: 80.5%
Validation accuracy: 78.2%
Minibatch loss at step 1500: 1.375561
Minibatch accuracy: 79.7%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 0.990266
Minibatch accuracy: 82.8%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 0.741617
Minibatch accuracy: 85.9%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.796009
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
Test accuracy: 89.0%


Adding regularization to network

In [10]:
batch_size = 128

num_hidden_layer_nodes = 1024

strength_of_regularizer = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables, since we got two layers, we got more weights and biases
  weights_layer_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_layer_nodes]))
  biases_layer_1 = tf.Variable(tf.zeros([num_hidden_layer_nodes]))
  weights_layer_2 = tf.Variable(tf.truncated_normal([num_hidden_layer_nodes, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  layer_1_nets = tf.matmul(tf_train_dataset, weights_layer_1) + biases_layer_1
  layer_1_activations = tf.nn.relu(layer_1_nets)
  logits = tf.matmul(layer_1_activations, weights_layer_2) + biases_layer_2
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Adding Regularization
  L2_regularizer = (tf.nn.l2_loss(weights_layer_1) + tf.nn.l2_loss(biases_layer_1) + 
                    tf.nn.l2_loss(weights_layer_2) + tf.nn.l2_loss(biases_layer_2))

  # Add the regularization term to the loss.
  loss += strength_of_regularizer * L2_regularizer  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_layer_1) + biases_layer_1), weights_layer_2) + biases_layer_2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_layer_1) + biases_layer_1), weights_layer_2) + biases_layer_2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3470.343506
Minibatch accuracy: 10.2%
Validation accuracy: 31.0%
Minibatch loss at step 500: 1159.185913
Minibatch accuracy: 79.7%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 422.229187
Minibatch accuracy: 78.1%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 155.056152
Minibatch accuracy: 81.2%
Validation accuracy: 84.7%
Minibatch loss at step 2000: 57.169643
Minibatch accuracy: 85.2%
Validation accuracy: 86.2%
Minibatch loss at step 2500: 21.400436
Minibatch accuracy: 89.1%
Validation accuracy: 86.5%
Minibatch loss at step 3000: 8.315651
Minibatch accuracy: 83.6%
Validation accuracy: 85.8%
Test accuracy: 91.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [22]:
batch_size = 128

num_hidden_layer_nodes = 1024

strength_of_regularizer = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables, since we got two layers, we got more weights and biases
  weights_layer_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_layer_nodes]))
  biases_layer_1 = tf.Variable(tf.zeros([num_hidden_layer_nodes]))
  weights_layer_2 = tf.Variable(tf.truncated_normal([num_hidden_layer_nodes, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  layer_1_nets = tf.matmul(tf_train_dataset, weights_layer_1) + biases_layer_1
  layer_1_activations = tf.nn.relu(layer_1_nets)
  logits = tf.matmul(layer_1_activations, weights_layer_2) + biases_layer_2
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  # Adding Regularization
  L2_regularizer = (tf.nn.l2_loss(weights_layer_1) + tf.nn.l2_loss(biases_layer_1) + 
                    tf.nn.l2_loss(weights_layer_2) + tf.nn.l2_loss(biases_layer_2))

  # Add the regularization term to the loss.
  loss += strength_of_regularizer * L2_regularizer  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_layer_1) + biases_layer_1), weights_layer_2) + biases_layer_2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_layer_1) + biases_layer_1), weights_layer_2) + biases_layer_2)

num_steps = 20

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    print("Minibatch loss at step %d: %f" % (step, l))
    print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
    print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 620.902466
Minibatch accuracy: 14.1%
Validation accuracy: 31.8%
Minibatch loss at step 1: 555.204102
Minibatch accuracy: 29.7%
Validation accuracy: 35.3%
Minibatch loss at step 2: 500.180145
Minibatch accuracy: 28.9%
Validation accuracy: 43.6%
Minibatch loss at step 3: 523.684692
Minibatch accuracy: 45.3%
Validation accuracy: 47.2%
Minibatch loss at step 4: 453.636597
Minibatch accuracy: 50.0%
Validation accuracy: 55.6%
Minibatch loss at step 5: 411.323425
Minibatch accuracy: 62.5%
Validation accuracy: 59.8%
Minibatch loss at step 6: 418.012390
Minibatch accuracy: 56.2%
Validation accuracy: 57.1%
Minibatch loss at step 7: 429.379242
Minibatch accuracy: 58.6%
Validation accuracy: 57.6%
Minibatch loss at step 8: 403.380310
Minibatch accuracy: 57.0%
Validation accuracy: 61.5%
Minibatch loss at step 9: 423.297455
Minibatch accuracy: 60.9%
Validation accuracy: 58.8%
Minibatch loss at step 10: 426.566956
Minibatch accuracy: 57.8%
Validation accuracy: 65.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [21]:
batch_size = 128

num_hidden_layer_nodes = 1024

strength_of_regularizer = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables, since we got two layers, we got more weights and biases
  weights_layer_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_layer_nodes]))
  biases_layer_1 = tf.Variable(tf.zeros([num_hidden_layer_nodes]))
  weights_layer_2 = tf.Variable(tf.truncated_normal([num_hidden_layer_nodes, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  layer_1_nets = tf.matmul(tf_train_dataset, weights_layer_1) + biases_layer_1
  layer_1_activations = tf.nn.dropout(tf.nn.relu(layer_1_nets), 0.5) # add dropout
  logits = tf.matmul(layer_1_activations, weights_layer_2) + biases_layer_2
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  # Adding Regularization
  L2_regularizer = (tf.nn.l2_loss(weights_layer_1) + tf.nn.l2_loss(biases_layer_1) + 
                    tf.nn.l2_loss(weights_layer_2) + tf.nn.l2_loss(biases_layer_2))

  # Add the regularization term to the loss.
  loss += strength_of_regularizer * L2_regularizer  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_layer_1) + biases_layer_1), weights_layer_2) + biases_layer_2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_layer_1) + biases_layer_1), weights_layer_2) + biases_layer_2)

num_steps = 20

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    print("Minibatch loss at step %d: %f" % (step, l))
    print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
    print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 795.764526
Minibatch accuracy: 10.9%
Validation accuracy: 17.9%
Minibatch loss at step 1: 767.735901
Minibatch accuracy: 12.5%
Validation accuracy: 35.2%
Minibatch loss at step 2: 672.971313
Minibatch accuracy: 27.3%
Validation accuracy: 44.7%
Minibatch loss at step 3: 628.213196
Minibatch accuracy: 33.6%
Validation accuracy: 49.5%
Minibatch loss at step 4: 584.371338
Minibatch accuracy: 38.3%
Validation accuracy: 57.8%
Minibatch loss at step 5: 498.990662
Minibatch accuracy: 50.8%
Validation accuracy: 60.8%
Minibatch loss at step 6: 493.289154
Minibatch accuracy: 48.4%
Validation accuracy: 67.5%
Minibatch loss at step 7: 484.635315
Minibatch accuracy: 47.7%
Validation accuracy: 68.1%
Minibatch loss at step 8: 449.798767
Minibatch accuracy: 55.5%
Validation accuracy: 71.5%
Minibatch loss at step 9: 444.690552
Minibatch accuracy: 58.6%
Validation accuracy: 69.8%
Minibatch loss at step 10: 517.569214
Minibatch accuracy: 53.1%
Validation accuracy: 67.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [38]:
batch_size = 128

num_hidden_layer_nodes = 1024

strength_of_regularizer = 0.005

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables, since we got two layers, we got more weights and biases
  weights_layer_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_layer_nodes]))
  biases_layer_1 = tf.Variable(tf.zeros([num_hidden_layer_nodes]))
  weights_layer_2 = tf.Variable(tf.truncated_normal([num_hidden_layer_nodes, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  layer_1_nets = tf.matmul(tf_train_dataset, weights_layer_1) + biases_layer_1
  layer_1_activations = tf.nn.dropout(tf.nn.relu(layer_1_nets),0.5)
  logits = tf.matmul(layer_1_activations, weights_layer_2) + biases_layer_2
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Adding Regularization
  L2_regularizer = (tf.nn.l2_loss(weights_layer_1) + tf.nn.l2_loss(biases_layer_1) + 
                    tf.nn.l2_loss(weights_layer_2) + tf.nn.l2_loss(biases_layer_2))

  # Add the regularization term to the loss.
  loss += strength_of_regularizer * L2_regularizer  
    
  # Optimizer. weight decay
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.88)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_layer_1) + biases_layer_1), weights_layer_2) + biases_layer_2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_layer_1) + biases_layer_1), weights_layer_2) + biases_layer_2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2091.604004
Minibatch accuracy: 10.9%
Validation accuracy: 28.4%
Minibatch loss at step 500: 151.270004
Minibatch accuracy: 69.5%
Validation accuracy: 81.8%
Minibatch loss at step 1000: 19.376905
Minibatch accuracy: 85.2%
Validation accuracy: 85.2%
Minibatch loss at step 1500: 3.722898
Minibatch accuracy: 82.0%
Validation accuracy: 85.7%
Minibatch loss at step 2000: 1.372488
Minibatch accuracy: 83.6%
Validation accuracy: 85.8%
Minibatch loss at step 2500: 0.751025
Minibatch accuracy: 85.9%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 0.701022
Minibatch accuracy: 85.9%
Validation accuracy: 85.9%
Test accuracy: 91.8%
